In [4]:
import gradio as gr
import os
from IPython.display import Markdown
from python_scripts import llm_rag, machine_translation, text_to_speech, whisper_setup, get_audio, utils

# Define paths dynamically
PATH = os.getcwd().replace('\\\\', '/')
root_path = PATH + '/Datasets/MeDAL'
data_path = root_path + '/training_data(2).jsonl'

In [5]:
def SMTS(audio):
    try:    
        # Step 2: Transcribe Query to English
        whisper_models = ["tiny", "base", "small", "medium", "large"]
        transcript = whisper_setup.transcribe_audio(audio, 'tiny')

        # Step 3: Feed query into the LLM
        models = { 
            'llama_ours': 'ubaidtariq8/llama2-med-genai',
            'llama_13b': 'a16z-infra/llama13b-v2-chat:df7690f1994d94e96ad9d568eac121aecf50684a0b0963b25a41cc40061269e5',
            'mixtral': 'mistralai/mixtral-8x7b-instruct-v0.1',
            'llama_70b': 'meta/llama-2-70b-chat:2796ee9483c3fd7aa2e171d38f4ca12251a30609463dcfd4cd76703f22e96cdf'
        }

        model = llm_rag.DocumentEmbeddingPipeline(model_version = models['llama_ours'], chroma_path = root_path)
        model.setup_environment()
        model.prepare_documents(data_path = data_path, collection_name = "medal_filtered", joining = True, persistent = False)
        model.embed_and_index()

        response = [model.query_data(query = transcript['text'])]

        # Step 4: Translate it back to the user language
        translated_text = machine_translation.translate_text(text = " ".join(response), src_lang = 'en', trg_lang = transcript['lang'])

        # Step 5: Now speak the response in the user's language
        # answer_path = utils.generate_answer_path(original_path = audio_path)
        audio_answer_path = data_path + '/audio.wav'
        text_to_speech.multilingual_text_to_speech(text = translated_text, filepath = audio_answer_path)

        display(Markdown(f"<b>{translated_text}</b>"))
        return display(Markdown(f"<b>{translated_text}</b>")), utils.play_wav(audio_answer_path)
    except Exception as e:
        print("An error occurred:", e)

In [6]:
demo = gr.Interface(
    SMTS,
    gr.Audio(sources = ["microphone"]),
    ["number", "audio"],
)

demo.launch()


Running on local URL:  http://127.0.0.1:7861

To create a public link, set `share=True` in `launch()`.


An error occurred: Invalid model size 't', expected one of: tiny.en, tiny, base.en, base, small.en, small, medium.en, medium, large-v1, large-v2, large-v3, large, distil-large-v2, distil-medium.en, distil-small.en


Traceback (most recent call last):
  File "c:\Users\Talha\AppData\Local\Programs\Python\Python311\Lib\site-packages\gradio\queueing.py", line 527, in process_events
    response = await route_utils.call_process_api(
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\Talha\AppData\Local\Programs\Python\Python311\Lib\site-packages\gradio\route_utils.py", line 261, in call_process_api
    output = await app.get_blocks().process_api(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\Talha\AppData\Local\Programs\Python\Python311\Lib\site-packages\gradio\blocks.py", line 1797, in process_api
    data = await self.postprocess_data(fn_index, result["prediction"], state)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\Talha\AppData\Local\Programs\Python\Python311\Lib\site-packages\gradio\blocks.py", line 1565, in postprocess_data
    self.validate_outputs(fn_index, predictions)  # type: ignore
    ^^^^^^^^^^^^^^^^^^